In [1]:
import pandas as pd
import geopandas as gp
import os

from shapely import wkt
from shapely.geometry import Point
from tqdm import tqdm

tqdm.pandas()

busstops = pd.read_csv('data/busstops_norway.csv')
busstops = gp.GeoDataFrame(busstops, geometry=busstops.geometry.apply(wkt.loads))

train = pd.read_csv('data/stores_train.csv')[['store_id', 'lon','lat']]#[0:20]
train = gp.GeoDataFrame(train, geometry=gp.points_from_xy(train.lon, train.lat)).drop(columns=['lon', 'lat'])
train = train.set_crs('epsg:4326', allow_override=True).to_crs('epsg:3857')

test = pd.read_csv('data/stores_test.csv')[['store_id', 'lon', 'lat']]  # [0:20]
test = gp.GeoDataFrame(test, geometry=gp.points_from_xy(test.lon, test.lat)).drop(columns=['lon', 'lat'])
test = test.set_crs('epsg:4326', allow_override=True).to_crs('epsg:3857')


In [2]:
gp.show_versions()


SYSTEM INFO
-----------
python     : 3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]
executable : c:\dev\maskin\maskinprosjekt\venv\Scripts\python.exe
machine    : Windows-10-10.0.22000-SP0

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.5.2
GDAL data dir: c:\dev\maskin\maskinprosjekt\venv\lib\site-packages\fiona\gdal_data
PROJ       : 9.1.0
PROJ data dir: c:\dev\maskin\maskinprosjekt\venv\lib\site-packages\pyproj\proj_dir\share\proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.12.1
numpy      : 1.23.4
pandas     : 1.5.1
pyproj     : 3.4.0
shapely    : 1.8.5.post1
fiona      : 1.8.22
geoalchemy2: None
geopy      : 2.2.0
matplotlib : 3.6.0
mapclassify: None
pygeos     : 0.13
pyogrio    : None
psycopg2   : None
pyarrow    : None
rtree      : None


In [3]:
busstops = busstops.set_crs('epsg:4326', allow_override=True)
busstops = busstops.to_crs('epsg:3857')
print(busstops.crs)
# busstops.plot()
busstops


epsg:3857


,busstop_id,stopplace_type,importance_level,side_placement,geometry
0,853cb081-cc32-4880-aa3e-26e96870d874,Plattform og lomme,Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT (728101.520 8640047.651)
1,156b052b-2771-497a-b4f4-97fed59e1aca,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT (656762.828 8431462.584)
2,7312a280-e14f-4b09-a421-02e8fe1bc63e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT (1199816.247 8384169.375)
3,d9cda2c7-355a-49c1-b56c-a33180d2a82e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT (1199818.130 8384192.832)
4,f803bcd3-182d-450b-bbb4-113c6ca885c2,NaN,Mangler viktighetsnivå,MIDDLE_RIGHT,POINT (1199820.134 8384216.279)
...,...,...,...,...,...
68390,12edd887-a122-44e2-896b-a5f663917f88,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,MIDDLE,POINT (651699.196 8445068.826)
68391,38dd0aee-b497-43c2-943b-51ab5c6d0eb7,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,MIDDLE,POINT (651650.730 8445056.162)
68392,3803f0fe-d6dc-4211-a44d-bbc3453af289,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,MIDDLE,POINT (1185500.476 9086180.311)
68393,5b177625-d0dc-40ae-b554-e97f7d400e24,Plattform og lomme,Lokalt knutepunkt,MIDDLE,POINT (1020155.419 8640402.161)


In [4]:
# train = train.drop(columns=['bus_stops_count', *busstops.importance_level.unique()])

In [8]:

def cond_merge(row):
    output_dict = {}
    gp.options.use_pygeos = True
    
    stops_with_dist = busstops[busstops.distance(row.geometry) < 1000]
    # stops_with_dist = busstops.sindex.query(row.buffer(1000))
    output_dict['bus_stops_count'] = len(stops_with_dist)
    
    output_dict.update(stops_with_dist.importance_level.value_counts().reindex(
        busstops.importance_level.unique(), fill_value=0
    ).to_dict())
    
    return output_dict

# train = train.join(train.progress_apply(lambda row: cond_merge(row), axis=1, result_type='expand'))
# train.to_parquet('derived_data/stores_bus_stops_lt_1km_train')

test = test.join(test.progress_apply(lambda row: cond_merge(row), axis=1, result_type='expand'))
test.to_parquet('derived_data/stores_bus_stops_lt_1km_test')

100%|██████████| 8577/8577 [09:38<00:00, 14.81it/s]


In [9]:
bus_data_train = gp.read_parquet('derived_data/stores_bus_stops_lt_1km_test')
bus_data_train

,store_id,geometry,bus_stops_count,Mangler viktighetsnivå,Standard holdeplass,Lokalt knutepunkt,Nasjonalt knutepunkt,Regionalt knutepunkt,Annen viktig holdeplass
0,914206820-914239427-717245,POINT (1200806.820 8379432.793),13,13,0,0,0,0,0
1,916789157-916823770-824309,POINT (1067358.318 8224059.672),7,0,7,0,0,0,0
2,913341082-977479363-2948,POINT (1216112.410 8391314.847),6,6,0,0,0,0,0
3,889682582-889697172-28720,POINT (1164523.323 9204180.098),8,2,6,0,0,0,0
4,997991699-998006945-417222,POINT (1200594.429 8382192.955),12,12,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
8572,917323003-917383529-844309,POINT (592866.806 8236703.826),5,5,0,0,0,0,0
8573,917353379-917411824-845904,POINT (636919.593 8175248.695),13,1,12,0,0,0,0
8574,917072302-917089248-833647,POINT (1242682.011 8383798.540),5,4,0,0,0,1,0
8575,916960557-916993161-829908,POINT (574585.237 8499663.975),7,7,0,0,0,0,0
